# Ensemble model registration

In [1]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

/opt/conda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import requests
req_ensemble = requests.Request()
req_ensemble1 = requests.Request()
req_ensemble2 = requests.Request()
req_ensemble1.json = {"payload":[[5.8, 2.8, 5.1, 2.4]]}
req_ensemble2.json = {"payload":[[5.8, 2.8, 5.1, 2.4]]}
req_ensemble.json = {"model1":{"payload":[[5.8, 2.8, 5.1, 2.4],
       [6.0 , 2.2, 4.0 , 1.0 ]]},"model2":{"payload":[[5.8, 2.8, 5.1, 2.4],
       [6.0 , 2.2, 4.0 , 1.0 ]]}}
print(req_ensemble.json)

{'model1': {'payload': [[5.8, 2.8, 5.1, 2.4], [6.0, 2.2, 4.0, 1.0]]}, 'model2': {'payload': [[5.8, 2.8, 5.1, 2.4], [6.0, 2.2, 4.0, 1.0]]}}


# Ensemble model Score function

In [3]:
import json
import os
@scoring_func
def score_ensemble(model_dict, request):
    model1Id = os.environ.get("model1")
    model2Id = os.environ.get("model2")
    #Replace with Model 1 ID
    model1=model_dict[model1Id]["model_object"]
    score1=model_dict[model1Id]["score_object"]
    #Replace with Model 2 ID
    model2=model_dict[model2Id]["model_object"]
    score2=model_dict[model2Id]["score_object"]
    request1=request.json["model1"]
    request2=request.json["model2"]
    import numpy as np
    import requests
    req_ensemble1 = requests.Request()
    req_ensemble2 = requests.Request()
    req_ensemble1.json=request1
    req_ensemble2.json=request2
    a=score1(model1,req_ensemble1)
    b=score2(model2,req_ensemble2)
    model1_prob = np.array(a)
    model2_prob = np.array(b)
    final_prob = model1_prob*0.5 + model2_prob*0.5
    return final_prob.tolist()

# Registering Ensemble Model

In [4]:
help(register_ensemble_model)

Help on function register_ensemble_model in module mosaicml.api:

register_ensemble_model(name, description, version_list, scoring_func, init_script=None)
    Register ensemble model to the mosaic ai server
    
    Args:
        name (string): name of the model
        description (string): description of the model
        version_list (list): list of versions of dependent models required for final prediction
        E.g [ "v1", "v2"]
        scoring_func (function): function to be used for scoring
        init_script(string):script provided by user
    Returns:
        dict



In [ ]:
#Replace version IDs of individual models
import os
version1Id = os.environ.get("version1")
version2Id = os.environ.get("version2")
version_list = [version1Id,version2Id]

In [ ]:
register_ensemble_model(name="first_ensemble_model_iris_33",description="testing",version_list=version_list,scoring_func=score_ensemble)